# **Lexicon**

In [1]:
import numpy as np
import pandas as pd

### Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### DB 연결
# !pip install pymysql
import pymysql

### DB에 저장
import sqlalchemy
from sqlalchemy import create_engine

### 실시간 주식가격 데이터
# !pip install finance-datareader
import FinanceDataReader as fdr

### 텍스트 분석
## KoNLPy
# 1) JAVA 설치, 2) Python 버전과 맞는 JPype1-py3 설치, 3) !pip install konlpy, 4) 설치 경로에서 jvm.py 파일 코드 67번 줄 주석 처리 
from konlpy.tag import Okt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
## FastText
# !pip install gensim
# !pip install fasttext
import fasttext
import fasttext.util
# Facebook 한국어 Embedding 모델 다운로드 → 한 번만 설치하면 됨
fasttext.util.download_model('ko', if_exists='ignore')
# 유사도 계산
from gensim import models

### 모델 학습 및 평가
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

### 모델 저장 및 로드
import joblib

### 기타
import datetime
from collections import Counter
import sys
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf

print('GPU', '사용 가능' if tf.config.experimental.list_physical_devices('GPU') else '사용 불가능')

GPU 사용 불가능


In [3]:
!nvidia-smi

Fri Sep 17 05:43:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 452.56       Driver Version: 452.56       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce MX450      WDDM  | 00000000:2D:00.0 Off |                  N/A |
| N/A   64C    P8    N/A /  N/A |    119MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Read KOSELF & Test Data**

### **① KOSELF 감성 어휘 사전**

In [4]:
# 블로그에서 가져온 기본적인 한국어 긍부정 텍스트 목록
with open('positive_words_self.txt', encoding='utf-8') as pos_blog:
    positive_blog = pos_blog.readlines()
positive_blog = [pos_blog.replace('\n', '') for pos_blog in positive_blog]
with open('negative_words_self.txt', encoding='utf-8') as neg_blog:
    negative_blog = neg_blog.readlines()
negative_blog = [neg_blog.replace('\n', '') for neg_blog in negative_blog]

# KOSELF 감성 어휘 사전
with open('KOSELF_pos.txt', encoding='utf-8') as pos:
    positive = pos.readlines()
positive = [pos.replace('\n', '') for pos in positive]
with open('KOSELF_neg.txt', encoding='utf-8') as neg:
    negative = neg.readlines()
negative = [neg.replace('\n', '') for neg in negative]

### **② 2019년 Samsung Data**

In [5]:
lexicon_2019 = pd.read_csv('../../../../Code/Data/Test/Stock-Year/2019_samsung.csv')

lexicon_2019.head(1)

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization
0,삼성전자,5930,매일경제,2019010100,"""韓기업 파괴적 혁신해야 성장…전담조직 회사 밖에 만들라""",http://news.mk.co.kr/newsRead.php?year=2019&no=70,◆ 2019신년기획 인터뷰 ◆\r\n\n\n 위 그래프는 크리스텐슨 교수가 매일경제...,2019-01-02,0,39400,39400,38550,38750,7847664,0.001292,1,0,신년 기획 인터뷰 그래프 크리스 텐슨 교수 매일경제 직접 그림 그래프 의미 과거 한...


In [6]:
lexicon_2019[lexicon_2019['text'].isna()]

,st_n,st_cd,news,datetime,title,url,text,date,time,Open,High,Low,Close,Volume,Change,UpDown,Extremely_Changed,Tokenization


In [7]:
lexicon_2019.dropna(axis=0, inplace=True)
lexicon_2019 = lexicon_2019.reset_index(drop=True)
lexicon_2019.isna().sum()

st_n                 0
st_cd                0
news                 0
datetime             0
title                0
url                  0
text                 0
date                 0
time                 0
Open                 0
High                 0
Low                  0
Close                0
Volume               0
Change               0
UpDown               0
Extremely_Changed    0
Tokenization         0
dtype: int64

## **Calculate Cosine Similarity**

In [8]:
# 이미 학습된 FastText 내장 한국어 모델
try:
    print(ko_model, '모델 로드 불필요')
except:
    ko_model = models.fasttext.load_facebook_model('cc.ko.300.bin')

In [9]:
limit_number = 15000
sys.setrecursionlimit(limit_number)

In [9]:
pos_lexicon_2019 = pd.DataFrame(columns=['date', 'news_num', 'KOSELF_pos_word', 'news_word', 'cosine_similarity', 'frequency'])   # 'frequency' 구현할 방법은 추후 생각...ㅠㅠ

# KOSELF_pos와의 Cosine Similarity 계산
for x in range(len(lexicon_2019['Tokenization'])):
    news_num = x+1
    for y in range(len(positive)):
        for z in range(len(list(set(lexicon_2019['Tokenization'][x].split())))):
            if (ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]) >= 0.5) and (ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]) != 1.0):
                freq = 0
                for w in range(len(lexicon_2019['Tokenization'][x].split())):
                    if lexicon_2019['Tokenization'][x].split()[w] == list(set(lexicon_2019['Tokenization'][x].split()))[z]:
                        freq += 1
                data = {
                    'date': lexicon_2019['date'][x],
                    'news_num': news_num,
                    'KOSELF_pos_word': positive[y],
                    'news_word': list(set(lexicon_2019['Tokenization'][x].split()))[z],
                    'cosine_similarity': ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z]),
                    'frequency': freq
                    }
                pos_lexicon_2019 = pos_lexicon_2019.append(data, ignore_index=True)
                print("***{0} Cosine Similarity between <{1}> & <{2}> : {3}".format(lexicon_2019['date'][x], positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z], ko_model.wv.similarity(positive[y], list(set(lexicon_2019['Tokenization'][x].split()))[z])))

***2019-01-02 Cosine Similarity between <가치> & <의미> : 0.5354709625244141
***2019-01-02 Cosine Similarity between <중요한> & <가장> : 0.5371254086494446
***2019-01-02 Cosine Similarity between <확보> & <마련> : 0.5137236714363098
***2019-01-02 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2019-01-02 Cosine Similarity between <성공> & <실패> : 0.7068131566047668
***2019-01-02 Cosine Similarity between <증대> & <급증> : 0.5051040649414062
***2019-01-02 Cosine Similarity between <증대> & <확보> : 0.5161436796188354
***2019-01-02 Cosine Similarity between <증대> & <감소> : 0.6517643928527832
***2019-01-02 Cosine Similarity between <증대> & <증가> : 0.6946426630020142
***2019-01-02 Cosine Similarity between <추진> & <계획> : 0.5205424427986145
***2019-01-02 Cosine Similarity between <강세> & <급락> : 0.507910966873169
***2019-01-02 Cosine Similarity between <개선> & <방안> : 0.5635414123535156
***2019-01-02 Cosine Similarity between <경신> & <기록> : 0.522990882396698
***2019-01-02 Cosine Similarity between <상회한> & <견조할>

In [10]:
pos_lexicon_2019

,date,news_num,KOSELF_pos_word,news_word,cosine_similarity,frequency
0,2019-01-02,1,가치,의미,0.535471,5
1,2019-01-02,1,중요한,가장,0.537125,5
2,2019-01-02,1,확보,마련,0.513724,32
3,2019-01-02,3,경신,기록,0.522991,2
4,2019-01-02,3,성공,실패,0.706813,1
...,...,...,...,...,...,...
36847,2020-01-02,8456,본격적인,본격,0.610438,1
36848,2020-01-02,8456,증대,급증,0.505104,11
36849,2020-01-02,8456,증대,감소,0.651764,1
36850,2020-01-02,8456,증대,증가,0.694643,1


In [11]:
pos_lexicon_2019.to_csv('../../../../Code/Data/Test/Stock-Year/pos_samsung_2019.csv', index=False)

In [12]:
a = list(set(list(pos_lexicon_2019['news_word'])))

a

['옥슬리법',
 '애플리캐이션',
 '해향후',
 '쎄미켐동진쎄미켐',
 '렙비츠키',
 '컨테이선',
 '권평오',
 '서형준',
 '두부과자',
 '렌털시장',
 '강력',
 '율량점',
 '단점',
 '텅쉰슈마',
 '위벤처스',
 '맥쿼리인프라',
 '황현호',
 '그린무브',
 '비밀보호법',
 '팀호완',
 '호캉스',
 '지성규',
 '포튜나탑코',
 '실패',
 '인멸된',
 '관담회',
 '레드브릭',
 '둥관시',
 '계산통계학',
 '특징',
 '급락',
 '일장일단',
 '피슬러',
 '아싸왓찐타찟',
 '안중근함',
 '웰바이오텍',
 '정예슬',
 '수러풀러스',
 '더싼딜',
 '플루오르화수소',
 '전개',
 '최고급',
 '설진훈',
 '물량입니',
 '프로그람작성',
 '엄윤설',
 '폴드봇',
 '펫용품',
 '정본부장',
 '고취',
 '펀디엑스',
 '우호',
 '생뚱맞',
 '벳이글벳',
 '맞소송',
 '아르티잔스',
 '텍사트인스',
 '할롱베이',
 '증대',
 '윤자민',
 '랙티브',
 '겹악재',
 '트랜션',
 '완판되자',
 '홋카이도대',
 '던츠컵',
 '아나목스',
 '얼핏보',
 '입관식',
 '강창진',
 '정찬',
 '달음박질',
 '초특가타임딜',
 '액티브펀드',
 '냉각관',
 '저하',
 '훈풍',
 '엔터테인트',
 '쯔앙룽르앙낏',
 '강세',
 '위딩스',
 '스태디아',
 '강인길',
 '유수빈',
 '미뎀랩',
 '신강근',
 '렌털료',
 '중인',
 '임미란',
 '렌탈매장',
 '푸미훙지구',
 '디지털신호',
 '둔촌역',
 '실증특례',
 '젬디알젬',
 '향상',
 '시너지',
 '쿨런트펌프',
 '래피드커브',
 '인입량',
 '증진',
 '앤팁스',
 '프터스',
 '문수영',
 '꼽습니',
 '소싱전',
 '윈하이텍',
 '급증',
 '타인응우옌',
 '옌빈공단',
 '슬림하',
 '유데나필',
 '전장시',
 '파리올림픽',
 '폄훼하거',
 '넥쏘차',
 '횡

In [13]:
b = list(set(list(pos_lexicon_2019[pos_lexicon_2019['cosine_similarity']>=0.7]['news_word'])))

b

['우세',
 '추천',
 '불황',
 '추진',
 '약세',
 '갈란츠',
 '단점',
 '최고점',
 '실패',
 '셀루메드셀루메드',
 '밑돈',
 '띠웠다',
 '어센던트',
 '쎄미켐동진쎄미켐']

In [14]:
c = list(set(list(pos_lexicon_2019[pos_lexicon_2019['cosine_similarity']>=0.65]['news_word'])))

c

['메디콕스메디콕스',
 '맞소송',
 '코위버코위버',
 '콘엘비세미콘',
 '시나가와구',
 '아르티잔스',
 '두완정',
 '것이므',
 '패너마시티비치',
 '리밸린싱',
 '밑돈',
 '국봉환',
 '쎄미켐동진쎄미켐',
 '피슬러',
 '불황',
 '세리프체',
 '갈란츠',
 '최고점',
 '어센던트',
 '증가',
 '브이프론트',
 '추진',
 '약세',
 '단점',
 '웃돌',
 '감소',
 '텍옵트론텍',
 '그린무브',
 '적극',
 '추천',
 '스포워셔',
 '지성규',
 '실패',
 '셀루메드셀루메드',
 '셰도보팅',
 '띠웠다',
 '우세']